# Problems 1 - bert_text_classification

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from datasets import load_dataset, Dataset, DatasetDict
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load JSON lines into a DataFrame
dataset_df = pd.read_json("dataset.txt", lines=True)

In [ ]:
dataset_df = dataset_df[['reviewText', 'overall']].copy()
dataset_df.head()

,reviewText,overall
0,They look good and stick good! I just don't li...,4
1,These stickers work like the review says they ...,5
2,These are awesome and make my phone look so st...,5
3,Item arrived in great time and was in perfect ...,4
4,"awesome! stays on, and looks great. can be use...",5


In [ ]:
dataset_df.overall.value_counts() # labels for classification

,count
overall,
5,5699
4,2251
3,1220
1,1059
2,771


In [ ]:
# number of class
number_of_class = dataset_df.overall.nunique()
number_of_class

5

In [ ]:
# Split your DataFrame
train_dataset = dataset_df.sample(n=10000, random_state=42)
test_dataset = dataset_df.drop(train_dataset.index)

# Convert each split to a Dataset
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

# Adjust labels to be 0-indexed for the model
train_dataset = train_dataset.map(lambda example: {'overall': example['overall'] - 1})
test_dataset = test_dataset.map(lambda example: {'overall': example['overall'] - 1})


# Combine into a DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": test_dataset
})

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer.model_max_length

512

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["reviewText"], padding="max_length", truncation=True, return_tensors="pt")

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'overall'])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print("Train dataset shape:", tokenized_dataset['train'].shape, "\nValidated dataset shape:", tokenized_dataset['validation'].shape)

Train dataset shape: (10000, 6) 
Validated dataset shape: (1000, 6)


In [ ]:
type(tokenized_dataset['train'][0]['input_ids'])

torch.Tensor

In [ ]:
# Load the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=number_of_class).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Prepare the DataLoader
# top 10000 rows for training
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=True, batch_size=32)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=32)


In [ ]:
# Evaluate the model
model.eval()

all_labels = []
all_predictions = []

for batch in tqdm(eval_dataloader):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["overall"]
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=1)
    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(predictions.cpu().numpy())

100%|██████████| 32/32 [00:29<00:00,  1.08it/s]


In [ ]:
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Accuracy: {accuracy}")

print(classification_report(all_labels, all_predictions))
print(confusion_matrix(all_labels, all_predictions))

Accuracy: 0.1
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       104
           1       0.10      0.58      0.17        66
           2       0.10      0.56      0.17       110
           3       0.00      0.00      0.00       219
           4       0.00      0.00      0.00       501

    accuracy                           0.10      1000
   macro avg       0.04      0.23      0.07      1000
weighted avg       0.02      0.10      0.03      1000

[[  0  37  67   0   0]
 [  0  38  28   0   0]
 [  0  48  62   0   0]
 [  0  90 129   0   0]
 [  0 161 340   0   0]]


In [ ]:
# Prepare the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Train the model
model.train()

epochs = 5

for epoch in range(epochs):
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["overall"]
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

100%|██████████| 313/313 [15:42<00:00,  3.01s/it]


In [ ]:
# Evaluate the model
model.eval()

all_labels = []
all_predictions = []

for batch in tqdm(eval_dataloader):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["overall"]
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=1)
    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(predictions.cpu().numpy())

100%|██████████| 32/32 [00:33<00:00,  1.04s/it]


In [ ]:
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Accuracy: {accuracy}")

print(classification_report(all_labels, all_predictions))
print(confusion_matrix(all_labels, all_predictions))

Accuracy: 0.63
              precision    recall  f1-score   support

           0       0.83      0.52      0.64       104
           1       0.31      0.48      0.38        66
           2       0.36      0.36      0.36       110
           3       0.47      0.50      0.48       219
           4       0.81      0.79      0.80       501

    accuracy                           0.63      1000
   macro avg       0.56      0.53      0.53      1000
weighted avg       0.66      0.63      0.64      1000

[[ 54  32  10   3   5]
 [  8  32  20   3   3]
 [  0  29  40  30  11]
 [  2   8  28 110  71]
 [  1   3  14  89 394]]
